In [1]:
import os
import cv2
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.model_selection import KFold

In [2]:
# Define the number of classes
num_classes = 10

# Define image size and number of channels
img_rows, img_cols, img_channels = 32, 32, 1

# Define the CNN-LSTM model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_rows, img_cols, img_channels)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
from keras.layers import Reshape
from keras.layers import TimeDistributed, LSTM, Dense
model.add(Flatten())
model.add(Reshape((8, 8, 64)))
model.add(TimeDistributed(Flatten()))
model.add(TimeDistributed(Dense(128)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
# Define the number of folds
K = 3

# Define the KFold object
kf = KFold(n_splits=K, shuffle=True)

# Define lists to store the accuracy and loss for each fold
acc_per_fold = []
loss_per_fold = []

# Load the dataset
def load_dataset(dataset_path):
    X = []
    y = []
    for digit in range(num_classes):
        digit_path = os.path.join(dataset_path, str(digit))
        for filename in os.listdir(digit_path):
            image_path = os.path.join(digit_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (img_rows, img_cols))
            X.append(image)
            y.append(digit)
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], img_rows, img_cols, img_channels)
    X = X.astype('float32')
    X /= 255
    y = np_utils.to_categorical(y, num_classes)
    return X, y

In [5]:
# Load train dataset
train_X, train_y = load_dataset('C:/Users/Nayan/Desktop/Project/DevanagariHandwrittenDigitDataset/Train')

def train_model(train_X, train_y):
    # Train the model on the training data
    model.fit(train_X, train_y, epochs=20, batch_size=64)

    # Save the trained model to disk
    model.save('model.h5')

train_model(train_X, train_y)

Epoch 1/20
266/266 [==============================] - 23s 63ms/step - loss: 0.5822 - accuracy: 0.7953
Epoch 2/20
266/266 [==============================] - 17s 65ms/step - loss: 0.1001 - accuracy: 0.9690
Epoch 3/20
266/266 [==============================] - 16s 60ms/step - loss: 0.0511 - accuracy: 0.9840
Epoch 4/20
266/266 [==============================] - 16s 60ms/step - loss: 0.0368 - accuracy: 0.9875
Epoch 5/20
266/266 [==============================] - 16s 61ms/step - loss: 0.0200 - accuracy: 0.9938
Epoch 6/20
266/266 [==============================] - 16s 60ms/step - loss: 0.0121 - accuracy: 0.9965
Epoch 7/20
266/266 [==============================] - 16s 60ms/step - loss: 0.0198 - accuracy: 0.9938
Epoch 8/20
266/266 [==============================] - 16s 60ms/step - loss: 0.0154 - accuracy: 0.9954
Epoch 9/20
266/266 [==============================] - 16s 61ms/step - loss: 0.0134 - accuracy: 0.9958
Epoch 10/20
266/266 [==============================] - 16s 60ms/step - loss: 0.015

In [6]:
def predict(image):
    # Load the saved model
    saved_model = load_model('model.h5')

    # Preprocess the input image
    image = cv2.resize(image, (img_rows, img_cols))
    image = image.reshape(1, img_rows, img_cols, img_channels)
    image = image.astype('float32')
    image /= 255

    # Get the predicted class label
    pred = saved_model.predict(image)
    label = np.argmax(pred)

    return label